In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path
import matplotlib.pyplot as plt
import os
import json
import gzip
from urllib.request import urlopen

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2020-07-16 12:54:03--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  7.48MB/s    in 7.8s    

2020-07-16 12:54:11 (5.58 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [ ]:
data = []
with gzip.open('reviews_Cell_Phones_and_Accessories_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
print(len(data))
print(data[0])

194439
{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'helpful': [0, 0], 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again", 'overall': 4.0, 'summary': 'Looks Good', 'unixReviewTime': 1400630400, 'reviewTime': '05 21, 2014'}


In [ ]:
reviews = pd.DataFrame.from_dict(data)
reviews['asin'].value_counts()[:5].sort_values(ascending=False)

B005SUHPO6    837
B0042FV2SI    694
B008OHNZI0    657
B009RXU59C    636
B000S5Q9CA    628
Name: asin, dtype: int64

In [ ]:
reviews = reviews[reviews['asin'] == 'B008OHNZI0']
review = reviews[['reviewerID','asin','reviewText']]

In [ ]:
review.head()

,reviewerID,asin,reviewText
107080,A3GOOSN0MSIUNL,B008OHNZI0,It has a great thicker feel to it then a gener...
107081,AD6JYS5M2JS3V,B008OHNZI0,Very good fit. Easy to install. No bubbles. I ...
107082,A3Y37V6NNXYO4,B008OHNZI0,Great screen protector I've only used one so f...
107083,A1DNZ6WFXF3JF2,B008OHNZI0,Bought these a few weeks back and wanted to wa...
107084,ATKV0DKVBDT10,B008OHNZI0,It's easy to install (if you bother to read in...


In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-ctlxl_oo
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-ctlxl_oo
     |████████████████████████████████| 245kB 2.8MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8759829 sha256=493f32245cfc27166cea637961fcf103f564a260b5759ec08635bfea6a170f37
  Stored in directory: /tmp/pip-ephem-wheel-cache-6450jf7d/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [ ]:
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
import re
import pke
import datetime
nltk.download('stopwords')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def extract_keyphrases(caption, n):
    extractor = pke.unsupervised.TextRank() 
    extractor.load_document(caption)
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=n, stemming=False)
    return(keyphrases)
a = datetime.datetime.now()    
review['Abstract_Keyphrases'] = review.apply(lambda row: (extract_keyphrases(row['reviewText'],1)),axis=1)
b = datetime.datetime.now()
review.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,reviewerID,asin,reviewText,Abstract_Keyphrases
107080,A3GOOSN0MSIUNL,B008OHNZI0,It has a great thicker feel to it then a gener...,"[(great fit, 0.5127160131695542)]"
107081,AD6JYS5M2JS3V,B008OHNZI0,Very good fit. Easy to install. No bubbles. I ...,"[(other screen protectors, 0.3076925076923066)]"
107082,A3Y37V6NNXYO4,B008OHNZI0,Great screen protector I've only used one so f...,"[(great screen protector, 0.3797468354424148)]"
107083,A1DNZ6WFXF3JF2,B008OHNZI0,Bought these a few weeks back and wanted to wa...,"[(perfect fit, 0.17155648818923408)]"
107084,ATKV0DKVBDT10,B008OHNZI0,It's easy to install (if you bother to read in...,"[(clear screen protector, 0.4724414661784721)]"


In [ ]:
print(b-a)

0:06:12.576741


In [ ]:
review[-10:]

,reviewerID,asin,reviewText,Abstract_Keyphrases,title_cluster
107727,A2AAFB5UY9KBFL,B008OHNZI0,Originally 2 stars (changed it later)So the sc...,"[(multi - colored tiny dots, 0.116762019831747...",647
107728,A3OGNO1H7IO3NH,B008OHNZI0,Screen protector went on nice and felt premium...,"[(little more practice, 0.1916940207308027)]",648
107729,A20R37WRPLUM1D,B008OHNZI0,"Fairly easy to install (well, as easy as any o...","[(anti - glare surface, 0.606024027883172)]",649
107730,A1CMVCY4ZCUJSV,B008OHNZI0,"Received it pretty quick, I have the Iphone 5c...","[(screen protectors, 0.726864500857648)]",650
107731,A39HZEFMA3GKI4,B008OHNZI0,The screen protector is a perfect fit. I follo...,"[(perfect fit, 0.4493874773271151)]",651
107732,A2LTYEYGKBYXRR,B008OHNZI0,Box contents:- 1x iPhone 5 RetinaShield Screen...,[(tech armors retinashield blue light filter s...,652
107733,A3DZ7UB4R8JW6B,B008OHNZI0,The privacy screen is made of a durable materi...,"[(low brightness setting, 0.1367205921596235)]",653
107734,A1H685WQ1L4F4T,B008OHNZI0,I bought a otterbox case for my iphone 5 and i...,"[(loyal tech armor costumer, 0.1407949366699391)]",654
107735,AFY3OKG2XR4L6,B008OHNZI0,"I still have a few bubbles, but it's gotta be ...","[(free shipping, 0.20083407557708374)]",655
107736,A3OKKZM41VOZK7,B008OHNZI0,These are really the worst screen protectors I...,"[(worst screen protectors, 0.39473688210526314)]",656


In [ ]:
tag = []
for index, rows in review.iterrows(): 
  my_list =[rows.Abstract_Keyphrases] 
  tag.append(my_list) 



In [ ]:
tags = []
for i in range(len(tag)):
  try:
    if len(tag[i][0][0][0]) > 10:
      tags.append(tag[i][0][0])
  except:
    pass
tags[:5]

[('other screen protectors', 0.3076925076923066),
 ('great screen protector', 0.3797468354424148),
 ('perfect fit', 0.17155648818923408),
 ('clear screen protector', 0.4724414661784721),
 ('facetime camera hole', 0.261795708237837)]

In [ ]:
df = pd.DataFrame(tags, columns=['Tags','Score'])
df

,Tags,Score
0,other screen protectors,0.307693
1,great screen protector,0.379747
2,perfect fit,0.171556
3,clear screen protector,0.472441
4,facetime camera hole,0.261796
...,...,...
536,tech armors retinashield blue light filter scr...,0.181933
537,low brightness setting,0.136721
538,loyal tech armor costumer,0.140795
539,free shipping,0.200834


In [ ]:
df = df.sort_values("Score", axis = 0, ascending = False, inplace=False)
df[:5]

,Tags,Score
399,fast shipping,1.000000
138,great price work,1.000000
241,good quality screen protector,0.963855
256,less air bubbles,0.952381
355,tech armor iphone screen protectors,0.943396


In [ ]:
reviews = reviews[['reviewerID','asin','reviewText']]

In [ ]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=a3db3a26599fc54ca77bc7fced82b3e0cef643fd799f986dfd80801e4d324039
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [ ]:
from rake_nltk import Rake,Metric
r = Rake(min_length=2, max_length=3,ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO)

In [ ]:
def rake_keyphrases(content , n):
  tag = []
  r.extract_keywords_from_text(content)
  tag = r.get_ranked_phrases_with_scores()
  return tag[:n]

reviews['Abstract_Keyphrases'] = reviews.apply(lambda row: (rake_keyphrases(row['reviewText'],1)),axis=1)

In [ ]:
reviews.head()

,reviewerID,asin,reviewText,Abstract_Keyphrases
107080,A3GOOSN0MSIUNL,B008OHNZI0,It has a great thicker feel to it then a gener...,"[(8.5, great thicker feel)]"
107081,AD6JYS5M2JS3V,B008OHNZI0,Very good fit. Easy to install. No bubbles. I ...,"[(4.0, screen protectors)]"
107082,A3Y37V6NNXYO4,B008OHNZI0,Great screen protector I've only used one so f...,"[(8.5, one seemed easier)]"
107083,A1DNZ6WFXF3JF2,B008OHNZI0,Bought these a few weeks back and wanted to wa...,"[(9.0, 2 year old)]"
107084,ATKV0DKVBDT10,B008OHNZI0,It's easy to install (if you bother to read in...,"[(8.5, read instructions ).)]"


In [ ]:
tag = []
for index, rows in reviews.iterrows(): 
  my_list =[rows.Abstract_Keyphrases] 
  tag.append(my_list) 

In [ ]:
tag[:5]

[[[(8.5, 'great thicker feel')]],
 [[(4.0, 'screen protectors')]],
 [[(8.5, 'one seemed easier')]],
 [[(9.0, '2 year old')]],
 [[(8.5, 'read instructions ).')]]]

In [ ]:
tags = []
for i in range(len(tag)):
  try:
    if len(tag[i][0][0][1]) > 10:
      tags.append(tag[i][0][0])
  except:
    pass
tags[:5]

[(8.5, 'great thicker feel'),
 (4.0, 'screen protectors'),
 (8.5, 'one seemed easier'),
 (8.5, 'read instructions ).'),
 (8.5, 'facetime camera hole')]

In [ ]:
df_rake = pd.DataFrame(tags, columns=['Score','Tag'])
df_rake

,Score,Tag
0,8.5,great thicker feel
1,4.0,screen protectors
2,8.5,one seemed easier
3,8.5,read instructions ).
4,8.5,facetime camera hole
...,...,...
581,9.0,regulating sleep cycles
582,9.0,macbook pro due
583,9.0,leave finger marks
584,4.0,push bubbles


In [ ]:
df_rake = df_rake.sort_values("Score", axis = 0, ascending = False, inplace=False)
df_rake[:5]

,Score,Tag
293,9.0,cut outs line
371,9.0,takes hold easier
367,9.0,new iphone 5s
366,9.0,new iphone 5c
365,9.0,original protective plastic
